In [13]:
from flask import Flask,request, url_for, redirect, render_template
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

In [14]:
app = Flask(__name__)

model=pickle.load(open('../data/LogisticRegression.sav','rb'))


In [15]:
coefs = model.named_steps.clf.coef_
feats = model.named_steps.vect.get_feature_names()

# take coefficients and feature names, sorted? 

dct = {"feats":feats,"coef":coefs[0,:].tolist()}

titles = ["Chemical Engineer","Data Scientist","Financial Analyst","Physician","Recruiter","Underwriter"]

d = dict()
for i,title in enumerate(titles):
    coefs[i,:]
    dct = {"feats":feats,"coef":coefs[i,:].tolist()}
    top_words_df = pd.DataFrame(dct)
    top_words_df["coef"] = top_words_df["coef"].abs()
    top_words_df = top_words_df.sort_values("coef",ascending = False)
    a = top_words_df.feats.values.tolist()[:100]
    b = top_words_df.coef.values.tolist()[:100]
    d[title] = list(zip(a,b))
    
d["Chemical Engineer"]

[('engineering', 5.879409329900316),
 ('chemical', 4.772230527109973),
 ('process', 3.3170099975447167),
 ('engineer', 2.9177911907499716),
 ('technical', 2.72317592577277),
 ('design', 2.485422113167202),
 ('project', 2.2659184878510863),
 ('equipment', 2.1310321492308395),
 ('material', 2.082081757675433),
 ('test', 1.9239590257566028),
 ('financial', 1.8718159275538013),
 ('data', 1.7160258895631872),
 ('development', 1.7084233359907925),
 ('site', 1.6487213284621804),
 ('quality', 1.4613771793319854),
 ('operation', 1.4165262465248585),
 ('production', 1.413731408712815),
 ('control', 1.3080397262092551),
 ('product', 1.2551448502240043),
 ('analytics', 1.2261691263380674),
 ('integrity', 1.1494763337432252),
 ('recruiting', 1.1393874820553684),
 ('safety', 1.1205959188484198),
 ('system', 1.0567872969536969),
 ('underwriting', 1.0296649353079061),
 ('improvement', 0.9264691327363297),
 ('company', 0.8894135331146487),
 ('clinical', 0.8768210908904158),
 ('finance', 0.8730571148514

In [16]:
from sklearn.pipeline import Pipeline

def predict(pipe,X_test):
    return pipe.predict(X_test)

def predict_one(pipe, desc):
    return pipe.predict(pd.Series([desc])).tolist()[0]

def predict_one_proba(pipe, desc):
    return pipe.predict_proba(pd.Series([desc]))

In [17]:
def remove_stopwords(stopWords, descriptions):
    cleaned_descriptions = []
    for description in descriptions:
        temp_list = []
        for word in description.split():
            if word not in stopWords:
                temp_list.append(word.lower())
        cleaned_descriptions.append(' '.join(temp_list))
    return np.array(cleaned_descriptions)

def remove_punctuation(descriptions):
    no_punct_descriptions = []
    for description in descriptions:
        description_no_punct = ' '.join(RegexpTokenizer(r'\w+').tokenize(description))
        no_punct_descriptions.append(description_no_punct)
    return np.array(no_punct_descriptions)

def get_wordnet_pos(word):
    # nltk.download()

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {'J': wordnet.ADJ,
               'N': wordnet.NOUN,
               'V': wordnet.VERB,
               'R': wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_descriptions(descriptions):
    cleaned_descriptions = []
    for description in descriptions:
        temp_list = []
        for word in description.split():
            cleaned_word = WordNetLemmatizer().lemmatize(word, get_wordnet_pos(word))
            temp_list.append(cleaned_word)
        cleaned_descriptions.append(' '.join(temp_list))
    return np.array(cleaned_descriptions)

def clean_descriptions(stopWords, descriptions):
    no_punct = remove_punctuation(descriptions)
    no_punct_sw = remove_stopwords(stopWords, no_punct)
    cleaned = lemmatize_descriptions(no_punct_sw)
    return cleaned

def get_representative_jobs(df, kmeans):
    cluster_centers = kmeans.cluster_centers_
    for cent in cluster_centers:
        print('\nCluster Represnetations')
        dist = euclidean_distances(cent.reshape(1,-1), tfidf)
        order = np.argsort(dist)
        for o in order[0][:5]:
            title = df['job_title'].iloc[o]
            print(title)

In [18]:
def clean_input(resume):
    resume = (resume.split(" "))
    stopWords = set(stopwords.words('english'))
    tokenize_remove_punct = RegexpTokenizer(r'\w+')
    lemma = WordNetLemmatizer()

    # Cleaning descriptions for both the whole dataset and CO only
    return clean_descriptions(stopWords, resume)

In [20]:
def suggested_keywords(resume,occupation):
# Generate list of key words for that class
    suggested_words,key_words = [],[]
    for i in range(len(d[occupation])):
        key_words.append(d[occupation][i][0])

# add to suggested words list if resume doesn't contain matching words
    for i,key_word in enumerate(key_words):
        if i > 15 and suggested_words == []:
            contains_top_10 = True
            return []
        if key_word not in resume:
            suggested_words.append(key_word)

# Look through list of needed words
# If any word stems begin with list of words to change, change it to a understandable output.
    start_of_words_to_change = ["financ","analy","engineer", "model",]
    for i,word in enumerate(suggested_words):
        for check in start_of_words_to_change:
            if word.startswith(check):
                if check == "financ":
                    suggested_words[i] = "Variations of the word: finance"

                elif check == "analy":
                    suggested_words[i] = "Variations of the word: analysis"

                else:
                    suggested_words[i] = f"Variations of the word: {check}"

    return suggested_words[:5]



NameError: name 'ce_resume' is not defined

In [22]:


@app.route('/')
def hello_world():
    
    return render_template("resume_reccomendations.html")

#Take in the resume and return my outputs
# Forms . request to get resume  !? 

@app.route('/predict',methods=['POST','GET'])
def predict():
    occupation = predict_one(model,clean_input(ce_resume))
    output='{0:.{1}f}'.format(prediction[0][1], 2)
  
    return render_template('resume_reccomendations.html', pred= 'Your resume is classified as a {}. Here are my improvements to your resume for this occupation: {}'.format(occupation,suggested_keywords(clean_input(ce_resume),occupation)))
        
        
if __name__ == '__main__':
    app.run(debug=True)
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with inotify reloader


SystemExit: 1

/home/aqeelali7/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
@app.route('/predict',methods=['POST','GET'])
def predict():
    int_features=[int(x) for x in request.form.values()]
    final=[np.array(int_features)]
    print(int_features)
    print(final)
    prediction=model.predict_proba(final)
    output='{0:.{1}f}'.format(prediction[0][1], 2)

    if output>str(0.5):
        return render_template('forest_fire.html',pred='Your Forest is in Danger.\nProbability of fire occuring is {}'.format(output),bhai="kuch karna hain iska ab?")
    else:
        return render_template('forest_fire.html',pred='Your Forest is safe.\n Probability of fire occuring is {}'.format(output),bhai="Your Forest is Safe for now")


if __name__ == '__main__':
    app.run(debug=True)